In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

In [148]:
from torchvision.datasets import ImageFolder





transform = transforms.Compose(
    [transforms.Grayscale(),
        transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])


dataset = ImageFolder(root='./archive/trainingSet/unorder', transform=transform,)


dataset_len = len(dataset)
val_len = 10000
train_set, val_set = torch.utils.data.random_split(dataset, [dataset_len-val_len,val_len])






train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True, num_workers=0)
validation_loader = torch.utils.data.DataLoader(val_set, batch_size=16, shuffle=True, num_workers=0)


In [149]:
dataset.class_to_idx

{'even': 0, 'odd': 1}

In [150]:
dataset_len

42000

In [155]:
class ANet(nn.Module):
    def __init__(self):
        super(ANet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 6, 3)
        self.conv3 = nn.Conv2d(6, 12, 3)
        self.fc1 = nn.Linear(12, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1,12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = ANet()

In [156]:
summary(net,(1, 32, 32), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 31, 31]              15
         MaxPool2d-2            [-1, 3, 15, 15]               0
            Conv2d-3            [-1, 6, 13, 13]             168
         MaxPool2d-4              [-1, 6, 6, 6]               0
            Conv2d-5             [-1, 12, 4, 4]             660
         MaxPool2d-6             [-1, 12, 2, 2]               0
            Linear-7                   [-1, 10]             130
            Linear-8                    [-1, 2]              22
Total params: 995
Trainable params: 995
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.00
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


In [157]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [160]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        inputs = inputs.to(torch.float32)
#         labels = labels.to(torch.float32)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 0: 
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for data in validation_loader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the 10000 test images: %d %%' % (
                100 * correct / total))

print('Finished Training')

[1,     1] loss: 0.00035
Accuracy of the network on the 10000 test images: 50 %
[1,   501] loss: 0.10805
Accuracy of the network on the 10000 test images: 89 %
[1,  1001] loss: 0.05922
Accuracy of the network on the 10000 test images: 92 %
[1,  1501] loss: 0.04916
Accuracy of the network on the 10000 test images: 92 %
[2,     1] loss: 0.00037
Accuracy of the network on the 10000 test images: 93 %
[2,   501] loss: 0.04266
Accuracy of the network on the 10000 test images: 93 %
[2,  1001] loss: 0.03804
Accuracy of the network on the 10000 test images: 94 %
[2,  1501] loss: 0.03424
Accuracy of the network on the 10000 test images: 95 %
[3,     1] loss: 0.00005
Accuracy of the network on the 10000 test images: 95 %
[3,   501] loss: 0.03595
Accuracy of the network on the 10000 test images: 95 %
[3,  1001] loss: 0.03191
Accuracy of the network on the 10000 test images: 95 %
[3,  1501] loss: 0.03337
Accuracy of the network on the 10000 test images: 94 %
[4,     1] loss: 0.00022
Accuracy of the

In [163]:
class FinalNet(nn.Module):
    def __init__(self):
        super(FinalNet, self).__init__()
        self.fc1 = nn.Linear(12, 10)

    def forward(self, x):
        x = x.view(-1,12)
        x = F.relu(self.fc1(x))
        return x


finalnet = FinalNet()
finalnet.to(device)

Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))

In [ ]:
class Ensemble(nn.Module):
    def __init__(self, net, FinalNet):
        super(Ensemble, self).__init__()
        self.modelA = net
        self.modelB = FinalNet
        
    def forward(self, x):
        # temp_output
        ae_output,x = self.modelA(x)
        # model output
        x1 = self.modelB(x)
        return x1,ae_output

ensemble_model = Ensemble(net, finalnet)
ensemble_model.to(device)

In [167]:
for param in net.parameters():
    print(param.shape)
#     param.requires_grad = False

torch.Size([3, 1, 2, 2])
torch.Size([3])
torch.Size([6, 3, 3, 3])
torch.Size([6])
torch.Size([12, 6, 3, 3])
torch.Size([12])
torch.Size([10, 12])
torch.Size([10])
torch.Size([2, 10])
torch.Size([2])


In [169]:
index=0
for param in net.parameters():
    if index <6:
        param.requires_grad = False
        index+=1
    

In [ ]:
index=0
for param in net.parameters():
    if index <2:
        param.requires_grad = False
        index+=1
    

In [ ]:
index=0
for param in net.parameters():
    if index == 2 or index==3:
        param.requires_grad = False
    index+=1
    